In [32]:
# from PIL import Image
# import os
# from pathlib import Path

# resize_folder = './images'
# resize_output = './res_out'

# # 確保輸出目錄存在
# Path(resize_output).mkdir(parents=True, exist_ok=True)

# for ff in os.listdir(resize_folder):
#     img_path = os.path.join(resize_folder, ff)

#     img = Image.open(img_path)
#     # Get the original dimensions
#     width, height = img.size
#     # Resize to 33.33% of the original size
#     img_resize = img.resize((width // 3, height // 3))
#     # Save the resized image
#     img_resize.save(os.path.join(resize_output, ff))


In [34]:
import cv2
import numpy as np
import os
from pathlib import Path
import json

sobel_gradient_y_mask = np.array(
   [[-1, 0, 1], 
    [-2, 0, 2], 
    [-1, 0, 1]])
sobel_gradient_x_mask = np.array(
   [[-1, -2, -1],
    [ 0,  0,  0], 
    [ 1,  2,  1]])
laplacian_mask = np.array(
   [[-1, -1, -1],
    [-1,  8, -1], 
    [-1, -1, -1]])
average_mask = np.array(
   [[0.11111111, 0.11111111, 0.11111111],
    [0.11111111, 0.11111111, 0.11111111],
    [0.11111111, 0.11111111, 0.11111111]])   # 0.1111111 means 1/9


img_folder_name = 'images' #Change to the folder you use
image_folder = './'+ img_folder_name+'/'
output_folder = './Sharpening/Sharpening_' + img_folder_name + '/'
error_json = 'error_image_' + img_folder_name
err_list = []

for filename in os.listdir(image_folder):
    img_path = os.path.join(image_folder,filename)
    img = cv2.imread(img_path)
    print("Processing " + filename)
    if img is None:
        print(f"Warning: File {filename} could not be read, skipping.")
        result_json = {
            "img_name": filename,
        }
        err_list.append(result_json)
        continue
    # print(img.shape)
    # print(img.shape[0] * img.shape[1])

    padding_img = np.zeros((img.shape[0]+2, img.shape[1]+2, 3))

    # print(padding_img.shape)
    width = img.shape[1]
    height = img.shape[0]
    # print(padding_img[1:(height+1), 1:(width+1), :].shape)
    padding_img[1:height+1, 1:width+1, :] = img

    padding_img[:,0:1,:] = padding_img[:,1:2,:]
    padding_img[:,width+1:width+2,:] = padding_img[:,width:width+1,:]
    padding_img[0:1,:,:] = padding_img[1:2,:,:]
    padding_img[height+1:height+2,:,:] = padding_img[height:height+1,:,:]

    # padding_img = padding_img.astype(np.uint8)


    edge_img = np.zeros(img.shape)
    edge_blur_img = np.zeros(img.shape)
    edge_blur_img_normalized = np.zeros(img.shape)
    laplacian_img = np.zeros(img.shape)
    sharped_img = np.zeros(img.shape)
    sharped_sobel_img = np.zeros(img.shape)

    # 計算 Laplacian 與 Sobel
    # 計算 Laplacian 與 Sobel
    for y in range(padding_img.shape[0]-2):
        for x in range(padding_img.shape[1]-2):
            
            subimg_r = padding_img[y:y+3, x:x+3, 0]
            subimg_g = padding_img[y:y+3, x:x+3, 1]
            subimg_b = padding_img[y:y+3, x:x+3, 2]
            
            # Sobel 
            gradient_r = abs(np.sum(subimg_r * sobel_gradient_y_mask)) + abs(np.sum(subimg_r * sobel_gradient_x_mask))
            gradient_g = abs(np.sum(subimg_g * sobel_gradient_y_mask)) + abs(np.sum(subimg_g * sobel_gradient_x_mask))
            gradient_b = abs(np.sum(subimg_b * sobel_gradient_y_mask)) + abs(np.sum(subimg_b * sobel_gradient_x_mask))
            
            edge_img[y, x, 0] = gradient_r
            edge_img[y, x, 1] = gradient_g
            edge_img[y, x, 2] = gradient_b
                
            # Laplacian
            laplacian_r = np.sum(subimg_r * laplacian_mask)
            laplacian_g = np.sum(subimg_g * laplacian_mask)
            laplacian_b = np.sum(subimg_b * laplacian_mask)
            
            laplacian_img[y, x, 0] = laplacian_r
            laplacian_img[y, x, 1] = laplacian_g
            laplacian_img[y, x, 2] = laplacian_b
            
            # print(y, x, laplacian_r)
            
        # print()


    # Laplacian Sharpening...        
    sharped_img = img + laplacian_img
    # print('sharped_img:', sharped_img)

    sharped_img[sharped_img > 255] = 255
    sharped_img[sharped_img < 0] = 0
    # print('sharped_img_rearange:', sharped_img)


    # Sobel Sharpening...        
    padding_img = np.zeros((img.shape[0]+2, img.shape[1]+2, 3))
    padding_img[1:height+1, 1:width+1, :] = edge_img

    # 對 sobel gradient 做 average blur
    for y in range(padding_img.shape[0]-2):
        for x in range(padding_img.shape[1]-2):
            
            subimg_r = padding_img[y:y+3, x:x+3, 0]
            subimg_g = padding_img[y:y+3, x:x+3, 1]
            subimg_b = padding_img[y:y+3, x:x+3, 2]
            
            avg_r = np.sum(subimg_r * average_mask)
            avg_g = np.sum(subimg_g * average_mask)
            avg_b = np.sum(subimg_b * average_mask)
            
            edge_blur_img[y, x, 0] = avg_r
            edge_blur_img[y, x, 1] = avg_g
            edge_blur_img[y, x, 2] = avg_b

            # print(y, x, avg_r)
        # print()

    edge_blur_img_normalized = cv2.normalize(edge_blur_img, None, 0, 1, cv2.NORM_MINMAX)  
    # print('edge_blur_img_normalized_1:\n', edge_blur_img_normalized)


    avgMtpLapl = laplacian_img * edge_blur_img_normalized   # 二階微分 乘以 一階微分平均模糊
    # print('avgMtpLapl:\n', avgMtpLapl)

    # Sobel Sharpening, 原圖加上blur後的edge
    sharped_sobel_img = img + avgMtpLapl

    edge_img = cv2.normalize(edge_img,None,0,255,cv2.NORM_MINMAX)        
    edge_img = edge_img.astype(np.uint8)

    edge_blur_img = cv2.normalize(edge_blur_img,None,0,255,cv2.NORM_MINMAX)        
    edge_blur_img = edge_blur_img.astype(np.uint8)

    laplacian_img = cv2.normalize(laplacian_img,None,0,255,cv2.NORM_MINMAX)        
    laplacian_img = laplacian_img.astype(np.uint8)
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)
        # os.makedirs(output_folder + "/Gradient")
        # os.makedirs(output_folder + "/Gradient_blur")
        # os.makedirs(output_folder + "/AvgMtpLapl")
        # os.makedirs(output_folder + "/Laplacian")
        # os.makedirs(output_folder + "/Sharped")
        # os.makedirs(output_folder + "/Sharped_sobel")
    # cv2.imshow('gradient', edge_img)
    # cv2.imwrite(output_folder + "Gradient/" + Path(filename).stem  + "_gradient.jpg", edge_img)

    # cv2.imwrite(output_folder + "Gradient_blur/" + Path(filename).stem + "_gradient_blur.jpg", edge_blur_img)

    # cv2.imwrite(output_folder + "AvgMtpLapl/" + Path(filename).stem + "_avgMtpLapl.jpg", avgMtpLapl)

    # # cv2.imshow('laplacian', laplacian_img)
    # cv2.imwrite(output_folder + "Laplacian/" + Path(filename).stem + "_laplacian.jpg", laplacian_img)

    # # cv2.imshow('sharped', sharped_img)
    # cv2.imwrite(output_folder + "Sharped/" + Path(filename).stem + "_sharped.jpg", sharped_img)

    # cv2.imshow('sharped_sobel', sharped_sobel_img)
    cv2.imwrite(output_folder + "sharped_" + Path(filename).stem + ".jpg", sharped_sobel_img)

    # cv2.waitKey(0)
    # cv2.destroyAllWindows()
with open(output_folder+'/'+error_json,'+a') as f:
    json.dump(err_list,f,indent=1)


Processing bird_6.jpg
Processing IMG_9859.JPG
Processing bird_5.jpg
Processing lenna.jpg
Processing butterfly.jpg
Processing bird_2.jpg
Processing bird_1.jpg
Processing flower_3.jpg
Processing bird_8.jpg
Processing IMG_7367_2.jpg
Processing bird_4.jpg
Processing flower_1.jpg
Processing bug_1.jpg
Processing cat.jpg
Processing bird_3.jpg
Processing bird_7.jpg
Processing IMG_7367.JPG
Processing flower_2.jpg
